In [1]:
import numpy as np
from collections import deque

In [2]:
class Queue:
    def __init__(self):
        self.queue = deque([])

    def isEmpty(self):
        return len(self.queue) == 0

    def push(self, item):
        self.queue.append(item)

    def pop(self):
        return self.queue.popleft()

    def peek(self):
        return self.queue[0]

    def size(self):
        return len(self.queue)

    
class Stack:
    def __init__(self):
         self.stack = deque([])

    def isEmpty(self):
        return len(self.stack) == 0

    def push(self, item):
        self.stack.append(item)

    def pop(self):
        return self.stack.pop()

    def peek(self):
        return self.stack[-1]

    def size(self):
        return len(self.stack)

In [3]:
class Board:
    # data: numpy array containing board data
    # parent: a reference to the parent node
    def __init__(self, data=None, parent=None):
        self.data = data
        self.parent = parent
    
    # build a random board of shape (n0, n1)
    def build(self, n0, n1):
        data = np.arange(n0*n1)
        np.random.shuffle(data)
        self.data = data.reshape(n0,n1)
        return self
    
    # return depth of current board
    def depth(self):
        if self.parent is not None:
            return self.parent.depth() + 1
        else:
            return 0


class TileGame:
    # Get the children of the current board
    def get_available_moves(board):
        available_moves = []
        for i in range(board.data.size):
            coords = np.where(board.data==i)
            if coords[0] > 0:
                available_moves.append(((coords[0],coords[1]),(coords[0]-1,coords[1])))
            if coords[0] < board.data.shape[0]-1:
                available_moves.append(((coords[0],coords[1]),(coords[0]+1,coords[1])))
            if coords[1] > 0:
                available_moves.append(((coords[0],coords[1]),(coords[0],coords[1]-1)))
            if coords[1] < board.data.shape[1]-1:
                available_moves.append(((coords[0],coords[1]),(coords[0],coords[1]+1)))
        return available_moves
  
    # Return a new board with the specified move applied
    def examine_move(board, move):
        data = board.data.copy()
        data[move[0][0],move[0][1]], data[move[1][0],move[1][1]] = data[move[1][0],move[1][1]], data[move[0][0],move[0][1]]
        return Board(data=data, parent=board)
    
    # Check if the specified board is in the goal state
    def is_goal_state(board):
        goal = np.reshape(np.arange(board.data.size),board.data.shape)
        return np.all(board.data==goal)
    
    # Returns the 'path' from the starting board state to goal board state
    def extract_solution(board):    
        solution = []
        solution.append(board)
        while board.parent is not None:
            board = board.parent
            solution.append(board)
        return solution[::-1]
    
    # Prints the 'path' returned by 'extract_solution(board)'
    def print_solution(solution):
        for board in solution:
            print(board.data)
            print()

In [4]:
# *** Game Instructions ***
# Available Moves: Any two adjacent squares can be swapped
# Goal State: All squares are in numerical order. For instance, given a 2x3 board the goal state is: [[0,1,2],[3,4,5]]

In [5]:
# TODO: Breadth-First Search
# Hint: Use the Queue or Stack Implimentations Provided
def bfs(board):
    ########## TODO: START OF YOUR CODE ##########
    ### NOTE: Your solution must use an extended set. That is, you should maintain
    ###       a list/set of board states that have already been extended. Board states
    ###       that have already been extended should not be extended again. Practically,
    ###       a board state is said to have been extended once its children have
    ###       been examined. Return the size of the extended set as the second
    ###       return parameter. The first return parameter should be your solution
    ###       (see the HINT below).
    ### HINT: To add a board state to the extended set, add board.data.tostring()
    ###       instead of adding the board object itself. (Why?)
    ### HINT: Use TileGame.extract_solution(board) to generate the corresponding
    ###       solution from the final board state

    board_set = set()
    search_queue = Queue()
    search_queue.push(board)
    while not search_queue.isEmpty():
        current_board = search_queue.pop()
        if TileGame.is_goal_state(current_board):
            return TileGame.extract_solution(current_board), len(board_set)
        elif current_board.data.tostring() not in board_set:
            for each in TileGame.get_available_moves(current_board):
                next_board = TileGame.examine_move(current_board, each)
                if next_board.data.tostring() not in board_set:
                    search_queue.push(next_board)
            board_set.add(current_board.data.tostring())

    ########## TODO: END OF YOUR CODE ##########
    raise Exception("Unreachable")


board = Board(data=np.array([[4, 2, 3], [0, 5, 1]]))
solution, nodes_expanded = bfs(board)
print('Nodes Expanded:', nodes_expanded)
print('Solution:')
TileGame.print_solution(solution)

# TODO: Is BFS optimal? (Yes / No)
# Answer: Yes


Nodes Expanded: 355
Solution:
[[4 2 3]
 [0 5 1]]

[[0 2 3]
 [4 5 1]]

[[0 2 1]
 [4 5 3]]

[[0 1 2]
 [4 5 3]]

[[0 1 2]
 [4 3 5]]

[[0 1 2]
 [3 4 5]]



In [6]:
# TODO: Depth-First Search
# Hint: Use the Queue or Stack Implimentations Provided
def dfs(board):
    ######### TODO: START OF YOUR CODE ##########
    ## NOTE: Your solution must use an extended set. That is, you should maintain
    ##       a list/set of board states that have already been extended. Board states
    ##       that have already been extended should not be extended again. Practically,
    ##       a board state is said to have been extended once its children have
    ##       been examined. Return the size of the extended set as the second
    ##       return parameter. The first return parameter should be your solution
    ##       (see the HINT below).
    ## HINT: To add a board state to the extended set, add board.data.tostring()
    ##       instead of adding the board object itself. (Why?)
    ## HINT: Use TileGame.extract_solution(board) to generate the corresponding
    ##       solution from the final board state
    board_set = set()
    search_stack = Stack()
    search_stack.push(board)
    while not search_stack.isEmpty():
        current_board = search_stack.pop()
        if TileGame.is_goal_state(current_board):
            return TileGame.extract_solution(current_board), len(board_set)
        elif current_board.data.tostring() not in board_set:
            for each in TileGame.get_available_moves(current_board):
                next_board = TileGame.examine_move(current_board, each)
                if next_board.data.tostring() not in board_set:
                    search_stack.push(next_board)
            board_set.add(current_board.data.tostring())

    ########## TODO: END OF YOUR CODE ##########
    raise Exception("Unreachable")
    
board = Board(data=np.array([[4,2,3],[0,5,1]]))
solution, nodes_expanded = dfs(board)
print('Nodes Expanded:', nodes_expanded)
print('Solution:')
TileGame.print_solution(solution)

# TODO: Is DFS optimal? (Yes / No)
# Answer: No

Nodes Expanded: 321
Solution:
[[4 2 3]
 [0 5 1]]

[[4 2 3]
 [0 1 5]]

[[4 2 5]
 [0 1 3]]

[[4 5 2]
 [0 1 3]]

[[5 4 2]
 [0 1 3]]

[[0 4 2]
 [5 1 3]]

[[0 4 2]
 [1 5 3]]

[[0 4 2]
 [1 3 5]]

[[0 4 5]
 [1 3 2]]

[[0 5 4]
 [1 3 2]]

[[5 0 4]
 [1 3 2]]

[[1 0 4]
 [5 3 2]]

[[1 0 4]
 [3 5 2]]

[[1 0 4]
 [3 2 5]]

[[1 0 5]
 [3 2 4]]

[[1 5 0]
 [3 2 4]]

[[5 1 0]
 [3 2 4]]

[[3 1 0]
 [5 2 4]]

[[3 1 0]
 [2 5 4]]

[[3 1 0]
 [2 4 5]]

[[3 1 5]
 [2 4 0]]

[[3 5 1]
 [2 4 0]]

[[5 3 1]
 [2 4 0]]

[[2 3 1]
 [5 4 0]]

[[2 3 1]
 [4 5 0]]

[[2 3 1]
 [4 0 5]]

[[2 3 5]
 [4 0 1]]

[[2 5 3]
 [4 0 1]]

[[5 2 3]
 [4 0 1]]

[[4 2 3]
 [5 0 1]]

[[2 4 3]
 [5 0 1]]

[[2 4 3]
 [0 5 1]]

[[2 4 3]
 [0 1 5]]

[[2 4 5]
 [0 1 3]]

[[2 5 4]
 [0 1 3]]

[[5 2 4]
 [0 1 3]]

[[0 2 4]
 [5 1 3]]

[[0 2 4]
 [1 5 3]]

[[0 2 4]
 [1 3 5]]

[[0 2 5]
 [1 3 4]]

[[0 5 2]
 [1 3 4]]

[[5 0 2]
 [1 3 4]]

[[1 0 2]
 [5 3 4]]

[[1 0 2]
 [3 5 4]]

[[1 0 2]
 [3 4 5]]

[[1 0 5]
 [3 4 2]]

[[1 5 0]
 [3 4 2]]

[[5 1 0]
 [3 4 2]]

[[3 1 0]
 

In [8]:
# TODO: Iterative Deepening Search
# Hint: Use the Queue or Stack Implimentations Provided
def ids(board):
    ########## TODO: START OF YOUR CODE ##########
    ### NOTE: Your solution must use an extended set. That is, you should maintain
    ###       a list/set of board states that have already been extended. Board states
    ###       that have already been extended should not be extended again. Practically,
    ###       a board state is said to have been extended once its children have
    ###       been examined. Return the size of the extended set as the second
    ###       return parameter. The first return parameter should be your solution
    ###       (see the HINT below).
    ### HINT: To add a board state to the extended set, add board.data.tostring()
    ###       instead of adding the board object itself. (Why?)
    ### HINT: Use TileGame.extract_solution(board) to generate the corresponding
    ###       solution from the final board state

    depth = 0
    board_list = set()
    while 1:
        board_list_inner = set()
        result = ids_inner(board, depth, board_list_inner)
        board_list = board_list.union(board_list_inner)
        if result:
            if TileGame.is_goal_state(result):
                return TileGame.extract_solution(result), len(board_list)
        depth += 1
    # ########## TODO: END OF YOUR CODE ##########
    raise Exception("Unreachable")
def ids_inner(board, depth, board_list):
    if TileGame.is_goal_state(board):
        return board
    if depth<=0:
        return
    if board.data.tostring() not in board_list:
        board_list.add(board.data.tostring())
        for each in TileGame.get_available_moves(board):
            result = ids_inner(TileGame.examine_move(board,each), depth - 1,board_list)
            if result:
                return result
    return

board = Board(data=np.array([[4, 2, 3], [0, 5, 1]]))
solution, nodes_expanded = ids(board)
print('Nodes Expanded:', nodes_expanded)
print('Solution:')
TileGame.print_solution(solution)

# TODO: Is IDS optimal? (Yes / No)
# Answer: Yes


Nodes Expanded: 124
Solution:
[[4 2 3]
 [0 5 1]]

[[0 2 3]
 [4 5 1]]

[[0 3 2]
 [4 5 1]]

[[0 3 2]
 [4 1 5]]

[[0 1 2]
 [4 3 5]]

[[0 1 2]
 [3 4 5]]

